# Inference - ImageNet

***

## Imports

In [1]:
import os
import glob
import scipy.io
import numpy as np
import tensorflow as tf
import larq_zoo as lqz

In [2]:
print('TensorFlow Version: {}'.format(tf.__version__))
print('Larq Zoo Version:   {}'.format(lqz.__version__))

TensorFlow Version: 2.4.0
Larq Zoo Version:   2.0.5


## Setting

In [4]:
# Top directory of validation dataset
image_dir = '/Volumes/NVMe SSD/datasets/imagenet/val'
if not os.path.exists(image_dir):
    print('[ERROR] Input image directory {} does not exist...'.format(image_dir))

## Load pretrained model

In [5]:
# Load pretrained model
model = lqz.sota.QuickNet(weights="imagenet")

## Preparation

In [6]:
# Ground truth
gt = list()
with open('ILSVRC2012_validation_ground_truth.txt', 'r') as f:
    for data in f.readlines():
        gt.append(int(data.replace('\n', '')))

In [7]:
meta = scipy.io.loadmat('meta.mat')
synsets = meta['synsets']

# WNID - ILSVRC2012_ID
keys = list()
vals = list()
for s in synsets:
    vals.append(s[0][1][0])
    keys.append(s[0][0][0][0])
    
d = dict(zip(keys, vals))

## Run inference

In [8]:
img_list = sorted(glob.glob(os.path.join(image_dir, '*.JPEG')))

In [10]:
acc_1 = list()
acc_5 = list()

for i, file in enumerate(img_list):
    # Load image
    img = tf.keras.preprocessing.image.load_img(file)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = lqz.preprocess_input(img)
    
    # Run inference
    preds = model.predict(np.expand_dims(img, axis=0))
    names = lqz.decode_predictions(preds, top=5)
    
    # Accuracy
    acc_1.append(d[gt[i]] == names[0][0][0])
    acc_5.append(d[gt[i]] in [item[0] for item in names[0]])

## Show result

In [11]:
print('----- Accuracy -----')
print('Top 1: {}'.format(np.sum(acc_1)/len(acc_1)))
print('Top 5: {}'.format(np.sum(acc_5)/len(acc_5)))

----- Accuracy -----
Top 1: 0.63298
Top 5: 0.84422
